## Utilizing Computer Vision for Object Detection in Meme Images

### Step 1: Importing the neccessary libraries

In [5]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import os
from collections import Counter

### Step 2: Loading the facebook/detr-resnet-50 Model for Object Detection



In [6]:
# Load DETR model
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

### Step 3: Setting Paths and category for Classification

In [7]:
# Define the base folder path
base_folder_path = 'C:\\Users\\soumi\\Desktop\\precog\\img\\validation_data'

# Define categories
categories = ["hateful_memes", "not_hateful_memes"]

### Step 4: Running the Pre-Trained Model on our Dataset

In [8]:
for category in categories:
    # Path to the current category
    current_folder_path = os.path.join(base_folder_path, category)

    # Output files for detection results and object frequency analysis
    output_file_path = f"output_results_{category}.txt"
    frequency_file_path = f"object_frequency_{category}.txt"

    # Object counter
    object_counter = Counter()

    with open(output_file_path, "w") as output_file:
        # Iterate over images in the current category folder
        for img_path in os.listdir(current_folder_path):
            full_img_path = os.path.join(current_folder_path, img_path)
            
            # Open and preprocess the image
            image = Image.open(full_img_path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt", input_data_format="channels_last")

            # Perform inference
            outputs = model(**inputs)

            # Convert outputs to COCO API format with threshold > 0.9
            target_sizes = torch.tensor([image.size[::-1]])
            results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

            # Write image path to output file
            output_file.write(f"Image - {full_img_path} :\n")

            # Write detection results to output file and update object counter
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                detected_object = model.config.id2label[label.item()]
                object_counter[detected_object] += 1
                box = [round(i, 2) for i in box.tolist()]
                output_file.write(f"Detected {detected_object} with confidence {round(score.item(), 3)} at location {box}\n")

            output_file.write("\n")
    
    # Write object frequency analysis to separate file
    with open(frequency_file_path, "w") as freq_file:
        for object, count in object_counter.most_common():
            freq_file.write(f"{object}: {count}\n")

    print(f"Detection results and object frequency analysis for {category} saved.")

Detection results and object frequency analysis for hateful_memes saved.
Detection results and object frequency analysis for not_hateful_memes saved.
